In [1]:
class ParseError(Exception):
    pass

In [2]:
import sys
sys.path.insert(0, 'sly/')

from sly import Lexer, Parser

class Lexer(Lexer):
    tokens = {A, B}
    ignore = ' \t'

    # Tokens
    A = r'[aA]'
    B = r'[bB]'

    # Ignored pattern
    ignore_newline = r'\n+'

    # Extra action for newlines
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')
    
    def error(self, t):
        raise ParseError()
        
class Parser(Parser):
    tokens = Lexer.tokens

    precedence = (
       ('left', A, B),
    )

    def __init__(self):
        self.names = { }
        
    def error(self, t):
        raise ParseError()
    
    # S <- S a S b S
    @_('statement A statement B statement')
    def statement(self, p):
        return (p[0], p[1], p[2], p[3], p[4])
    
    # S <- S b S a S
    @_('statement B statement A statement')
    def statement(self, p):
        return (p[0], p[1], p[2], p[3], p[4])
    
    # S <- ''
    @_('')
    def statement(self, p):
        return ()

In [3]:
lexer = Lexer()
parser = Parser()

def is_valid(text, min_length=5):
    if len(text) < min_length:
        return False

    try:
        return isinstance(parser.parse(lexer.tokenize(text)), tuple)
    except ParseError:
        return False


In [4]:
is_valid('abc')

False

In [5]:
import torch.nn as nn
import torch

class RNNDec(nn.Module):
    def __init__(self,
                 input_size=4,
                 noise_size=4,
                 hidden_size=20,
                 vocab_size=4):

        super().__init__()
        
        self.input_size = input_size
        self.noise_size = noise_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.rnn_dec = nn.RNN(input_size=self.input_size + self.noise_size, hidden_size=self.hidden_size, num_layers=1)
        
        self.lin_output = nn.Linear(self.hidden_size, self.vocab_size)
        self.embed = nn.Embedding(self.vocab_size, self.input_size)
        
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, x, noise, h):
        x = self.embed(x).reshape(1, 1, -1) # 1 x 1 x input_size
        x = self.relu(x)
        
        x = torch.cat((x, noise), dim=2) # 1 x 1 x noise_size + input_size
        
        dec_out, dec_hidden = self.rnn_dec(x, h)
        dec_out = self.relu(dec_out)
        
        logits = self.lin_output(dec_out.squeeze(0))
        
        return logits, dec_hidden


In [6]:
# 0 SOS
# 1 EOS
# 2 a
# 3 b

In [7]:
from torch import optim

lstm = RNNDec()
opt_dec = optim.Adam(lstm.parameters(), lr=1e-3)

ce_loss = nn.CrossEntropyLoss()

In [8]:
import torch.nn.functional as F

def model_forward(noise, sos_id = 0, eos_id = 1, sample=True):
    all_logits = []
    pred_ids = []
    
    prev = torch.tensor(sos_id)
    h = torch.zeros((1, 1, 20))
    for i in range(20):
        logits, h = lstm(prev, noise, h)
        
        if not sample:
            prev = torch.argmax(logits, dim=1)[0]
        else:
            probs = F.softmax(logits, dim=1)
            prev = torch.multinomial(probs[0], 1)[0]
        
        pred_ids.append(prev)

        all_logits.append(logits)
        
        if prev == eos_id:
            break
    
    return pred_ids, all_logits

In [9]:
import pickle

with open('train.pkl', 'rb') as file_in:
    train_data = pickle.load(file_in)

In [10]:
char_to_id = {
    'a': 2,
    'b': 3,
    '!': 1,
    '^': 0
}

id_to_char = {
    0: '^',
    1: '!',
    2: 'a',
    3: 'b'
}

In [11]:
MAX_SAMPLE = 1000
MIN_LENGTH = 0

In [12]:
from tqdm import tqdm

for epoch_idx in range(100):
    pbar = tqdm(train_data)
    for iter_idx, (noise_x, _) in enumerate(pbar):
        noise_x = torch.tensor(noise_x).reshape(1, 1, -1).float()
        
        sampled_label = None
        
        sample_count = 0 
        while sampled_label is None or not is_valid(sampled_label.replace('!', ''), min_length=MIN_LENGTH):
            if sample_count >= MAX_SAMPLE:
                print('WARNING: exceeded max sample number')
                break

            with torch.no_grad():
                s_pred_ids, _ = model_forward(noise_x, sample=True)

            sampled_label = ''.join([id_to_char[p.item()] for p in s_pred_ids])
        
            sample_count += 1
            
            #print('sampled: %s, is_valid: %s' % (sampled_label, is_valid(sampled_label.replace('!', ''), min_length=MIN_LENGTH)))
        
        pred_ids, all_logits = model_forward(noise_x, sample=False)
        
        loss = 0.0
        for i in range(min(len(all_logits), len(sampled_label))):
            logits = all_logits[i]
            y = sampled_label[i]
            pred = pred_ids[i]

            loss_single = ce_loss(logits, torch.tensor([char_to_id[y]], dtype=torch.long))
            
            loss += loss_single

        if iter_idx % 20 == 0:
            pbar.set_description("loss %s" % str(loss.item()))
        
        if iter_idx % 100 == 0:
            print('\nTARGET %s | IS_VALID %s' % (sampled_label, is_valid(sampled_label.replace('!', ''), min_length=MIN_LENGTH)))
            
            pred_str = ''.join([id_to_char[p.item()] for p in pred_ids])
            
            print('PRED %s | IS_VALID %s' % (pred_str, is_valid(pred_str, min_length=MIN_LENGTH)))
            print('loss', loss.item())
        
        loss /= len(all_logits)

        loss.backward()

        opt_dec.step()

        opt_dec.zero_grad()
    
    print()

loss 7.003281116485596:   0%|          | 3/5000 [00:00<08:20,  9.99it/s]


TARGET abba! | IS_VALID True
PRED bbbbbbbbbbbbbbbbbbbb | IS_VALID False
loss 7.003281116485596


loss 5.711129665374756:   2%|▏         | 105/5000 [00:03<01:47, 45.66it/s]


TARGET bbaa! | IS_VALID True
PRED bbbbbbbbbbbbbbbbbbbb | IS_VALID False
loss 5.711129665374756


loss 7.0570902824401855:   4%|▍         | 212/5000 [00:05<01:10, 68.04it/s]

...

loss 0.02486608736217022:  22%|██▏       | 1100/5000 [01:37<2:17:15,  2.11s/it]

loss 0.0021773900371044874:  22%|██▏       | 1101/5000 [01:39<2:19:19,  2.14s/it]


TARGET aaaaaaabbbbbbbbbbbbb | IS_VALID False
PRED aaaaaaabbbbbbbbbbbbb | IS_VALID False
loss 0.0021773900371044874


loss 0.0021773900371044874:  22%|██▏       | 1101/5000 [01:40<05:55, 10.97it/s]  


KeyboardInterrupt: 